# Compare data
compare the data created by `merge_files.ipynb` and exported from Charon4 with the data from sciebo repository eduardschacht.
The data is from 2019.

In the comparison an error=0.011 is allowed. The data from sciebo is rounded to the second digit. 

I copied the data created with `merge_files.ipynb` to `sciebo\DTS Data\Alsdorf\Daten\DTS_unprocessedDaten_Mario_oder_alt\compiled_data_desktop-folder`.

## Outdated
this script is not needed anymore because the data in `Daten_Mario_oder_alt` is the same as in the used database.
I keep it here if someone wants to check that again.

In [ ]:
# Imports
import pandas as pd
import glob

from my_func_mvw.functions import get_abspath

# User-Defined-Functions for Data import
def import_data_Daniel(path):
    """import for data from sciebo Eduardschacht repository"""
    df = pd.read_csv(path,delimiter = ',',index_col=0, header=7 )
    df = df.drop(df.columns[[0,1]], axis=1)
    df.index=pd.to_datetime(df.index, dayfirst = True).tz_localize(None)
    df.index=df.index.astype(str) # same column format as other dataframe
    df=df.transpose() # for having the same format as the other dataframe
    df.index=df.index.astype(float) #make depth to floats
    df.index.names = ['Depth [m]']
    return df

def import_merge_files_output(path):
    """imports the data created with the merge_files script"""
    df2=pd.read_csv(path,index_col=0)
    df2 = df2.round(decimals=2) #same number of decimal places as other dataframe
    return df2

def put_files_in_dataframe(list_of_paths,import_function):
    """
    merges files in one dataframe using an import_function (user-defined-functiom) in a list_of paths (list)
    """
    # Import Data
    dic={}
    for path in list_of_paths:
        dic[path]=import_function(path)

    # Merge files into one dataframe
    dic_concat=pd.concat(dic.values(),axis=1)
    
    return dic_concat

In [ ]:
# Import Data

## Data Paths
paths_to_working_dir=r"..\Alsdorf\Daten\unprocessed"
### daniel
daniel_ch1_paths=get_abspath(paths_to_working_dir + r"\Betrieb2019\Ch1\*")
daniel_ch2_paths=get_abspath(paths_to_working_dir + r"\Betrieb2019\Ch2\*")
daniel_ch3_paths=get_abspath(paths_to_working_dir + r"\Betrieb2019\Ch3\*")
daniel_ch4_paths=get_abspath(paths_to_working_dir + r"\Betrieb2019\Ch4\*")
### merge (other date period than daniel)
merge_ch1_paths=get_abspath(paths_to_working_dir + r"\Daten_Mario_oder_alt\compiled_data_desktop-folder\temp_ch1_2019_*")
merge_ch2_paths=get_abspath(paths_to_working_dir + r"\Daten_Mario_oder_alt\compiled_data_desktop-folder\temp_ch2_2019_*")
merge_ch3_paths=get_abspath(paths_to_working_dir + r"\Daten_Mario_oder_alt\compiled_data_desktop-folder\temp_ch3_2019_*")
merge_ch4_paths=get_abspath(paths_to_working_dir + r"\Daten_Mario_oder_alt\compiled_data_desktop-folder\temp_ch4_2019_*") 


In [ ]:
## Import files and merge them in one dataframe
### Channel 1
daniel_ch1=put_files_in_dataframe(daniel_ch1_paths,import_data_Daniel)
merge_ch1=put_files_in_dataframe(merge_ch1_paths,import_merge_files_output)
### Channel 2
daniel_ch2=put_files_in_dataframe(daniel_ch2_paths,import_data_Daniel)
merge_ch2=put_files_in_dataframe(merge_ch2_paths,import_merge_files_output)
### Channel 3
daniel_ch3=put_files_in_dataframe(daniel_ch3_paths,import_data_Daniel)
merge_ch3=put_files_in_dataframe(merge_ch3_paths,import_merge_files_output)
### Channel 4
daniel_ch4=put_files_in_dataframe(daniel_ch4_paths,import_data_Daniel)
merge_ch4=put_files_in_dataframe(merge_ch4_paths,import_merge_files_output)


# Drop columns which are in merge but not in daniel
# These are detected during the comparisson of the dataframes.
# This results in an error of the code, so these are excluded. 
# It also means, that these are new data!!!
merge_ch3=merge_ch3.drop(columns=['2019-03-31 21:59:31'])

In [ ]:
#daniel_ch1

In [ ]:
#merge_ch1

In [ ]:
# User-Defined-Functions for comparing the dataframes

def number_of_False_in_List(list_bool):
    """
    input: List of True and False
    returns: list of Index for all False entries
    """
    n_True=sum(list_bool) #number of Trues
    n_False=len(list_bool)-n_True #number of False
    
    # create index of all False entries
    index_False=[]
    for i, j in enumerate(list_bool):
        if j == False:
            index_False.append(i)
    return index_False, n_False


def compare_dataframes(df1_merge=merge_ch1,df2_daniel=daniel_ch1):
    """
    compare both dataframes if the values are equal

    df1_merge has to the merge dataframe, because it contains less data
    merge has less columns (less data), so it will be compared if all merge data is also in daniel data
    and if these are equal
    """
    # the print assumes that the data is sorted, which is not necessary the case
    #print("compare %.20s to %.20s of from the merged data with daniels data from sciebo"%(df1_merge.columns.min(), df1_merge.columns.max()))

    # check if all columns of merge are also in daniel dataframe
    #list of columns names which are in both
    columns_both=[i for i in df1_merge.columns if i in df2_daniel.columns]
    if set(df1_merge.columns).issubset(df2_daniel.columns):
        print("merge columns are a subset of daniel columns")
    else: #if this happens the following code does not work
        print("the following columns of merge are not in daniel")
        columns_not_in_daniel = [x for x in df1_merge.columns if x not in df2_daniel.columns]
        print(columns_not_in_daniel)
    
    # compare all values of the columns
    all_checks=[]
    for column in df1_merge.keys():
        #list of True False: each value of one columns is compared, based on the index, 
        #with the other dataframe column
        check=df1_merge[column]==df2_daniel[column]
        a=all(check) #returns True if all elements are True
        all_checks.append(a) # contains one entry for each column
    print("all columns are equal: %.10s"%all(all_checks))

    if all(all_checks)==False: #some values are not equal
        #Here the difference is calculated, when the difference is smaller than error it is ok.
        #an error of 0.01 could be somehow due to rounding. Did not further investigate why this difference exists
        #I think that is not very important
        error=0.011

        index_false, n_False=number_of_False_in_List(all_checks) # index of columns which are not completly equal
        print("number of False in list: %.0f"%n_False)

        print("check all False values and calculate difference")
        counter_exceeded_error=0
        for column_number in index_false: # go through all columns where False values are
            # check where in the column are the False values
            check=df2_daniel[df1_merge.keys()[column_number]]==df1_merge[df1_merge.keys()[column_number]]
            depth_false, n_False=number_of_False_in_List(check) # extract depth for all False values in this column
            #print("number of False in list: %.0f"%n_False)

            for depth in depth_false: # for each false depth, calculate the difference of the Temp.
                diff=df1_merge[df1_merge.keys()[column_number]][depth] - df2_daniel[df1_merge.keys()[column_number]][depth]

                if diff > error or diff < -1*error:
                    print("-----WARNING-----")
                    print("in %.0f at length %.0f the difference is larger than %.2f"%(column_number,depth,error))
                    print(diff)
                    print()
                    counter_exceeded_error+=1
        if counter_exceeded_error==0:
            print("all differences are in the range of %.4f"%error)
        print()

In [ ]:
# Compare dataframes
print("CH1")
compare_dataframes(merge_ch1,daniel_ch1)
print("CH2")
compare_dataframes(merge_ch2,daniel_ch2)
print("CH3")
compare_dataframes(merge_ch3,daniel_ch3)
print("CH4")
compare_dataframes(merge_ch4,daniel_ch4)

# Compare daniel Data with Charon4 data exported by me

In [ ]:
def import_charon4_txt(filename):
    """changed it druing repository structure change, not sure if it works as it expected; not tested"""
    save_to = "..\Alsdorf\Daten\my_database\csv" # outdated
    df=pd.read_csv(save_to + filename, index_col=0)
    return df

# temp_ch1_2021.csv temp_ch2_2021.csv temp_ch3_2021.csv temp_ch4_2021.csv
# temp_ch1_2020.csv temp_ch2_2020.csv temp_ch3_2020.csv temp_ch4_2020.csv
# temp_ch1_2019.csv temp_ch2_2019.csv temp_ch3_2019.csv temp_ch4_2019.csv
#filename=f"\\temp_ch2_2020.csv" 
data_charon4_2019={}
for channel in ["1","2","3","4"]:
    data_charon4_2019[channel]=import_charon4_txt(f"\\temp_ch{channel}_2019.csv")
    # make it the same format as the other dataframes in this notebook
    data_charon4_2019[channel].index.names = ['']
    data_charon4_2019[channel]=data_charon4_2019[channel].transpose()
    data_charon4_2019[channel].index.names = ['Depth [m]']
    data_charon4_2019[channel].index=data_charon4_2019[channel].index.astype(float)

In [ ]:
# now it is smaller than daniel data, and all of this data should be in daniel data
subset_data_charon4_2019_ch1=data_charon4_2019["1"][data_charon4_2019["1"].columns[5000:]]
subset_data_charon4_2019_ch2=data_charon4_2019["2"][data_charon4_2019["2"].columns[5000:]]
subset_data_charon4_2019_ch3=data_charon4_2019["3"][data_charon4_2019["3"].columns[5000:]]
subset_data_charon4_2019_ch4=data_charon4_2019["4"][data_charon4_2019["4"].columns[5000:]]

print("ch1")
compare_dataframes(df1_merge=subset_data_charon4_2019_ch1,df2_daniel=daniel_ch1)
print();print("ch2")
compare_dataframes(df1_merge=subset_data_charon4_2019_ch2,df2_daniel=daniel_ch2)
print();print("ch3")
compare_dataframes(df1_merge=subset_data_charon4_2019_ch3,df2_daniel=daniel_ch3)
print();print("ch4")
compare_dataframes(df1_merge=subset_data_charon4_2019_ch4,df2_daniel=daniel_ch4)